### Proyecto Individual Henry
**1_Extraction**

**Autor: Bioing. Urteaga Facundo Nahuel**

**Resumen:** Se realiza la carga de las bases de datos de steam. Cada archivo contiene su propia función de carga. Esto se debe a las particularidades que posee cada archivo que solicitan una corrección personalizada, perdiendo practicidad la creación de una función general de carga que contemple todas las correcciones. Este script comprende las siguientes etapas:

1. **Carga de librerías**
2. **Obtención de directorios de datos**
3. **Creación de funciones de carga**
4. **Ejecución de funciones de carga**
5. **Guardado de archivos en formato .csv**

### 1. Carga de librerías

In [1]:
# 1. Carga de librerías

import pandas as pd
import numpy as np
import os
import gzip
import json
import re

### 2. Obtener directorios de datos

In [2]:
# 2. Obtener directorios de datos

directorio_actual = os.getcwd()

# Construye las rutas completas al archivo usando os.path.join()

ruta_users_items = os.path.join(directorio_actual, "Datos", "users_items.json.gz")
ruta_user_reviews = os.path.join(directorio_actual, "Datos", "user_reviews.json.gz")
ruta_steam_games = os.path.join(directorio_actual, "Datos", "steam_games.json.gz")


### 3. Creación de funciones de carga
### 4. Ejecución de funciones de carga

In [3]:
# 3.1. Función para abrir el archivo users_items

def abrir_users_items(archivo):
    data_list = []
    aux = 0

    with gzip.open(archivo, 'rt', encoding='utf-8') as file:
        for line in file:
            try:
                line = line.strip()
                # Buscar todas las ocurrencias de 'item_name'
                ocurrencias_item_name = [i.start() for i in re.finditer("'item_name':", line)]
                # Reemplazar temporalmente las comillas simples en nombres de VJ por "*"
                for indice_item_name in ocurrencias_item_name:
                    #print(ocurrencias_item_name[indice_item_name])
                    inicio_nombre = indice_item_name + 14
                    #print(inicio_nombre)
                    fin_nombre = line.find(",", inicio_nombre)-1
                    #print(fin_nombre)
                    #print(line[inicio_nombre:fin_nombre])
                
                    if line.find("\"", inicio_nombre, fin_nombre):
                        line = line[:inicio_nombre] + line[inicio_nombre:fin_nombre].replace("\"", "*") + line[fin_nombre:]
                    if line.find("\\", inicio_nombre, fin_nombre):
                        line = line[:inicio_nombre] + line[inicio_nombre:fin_nombre].replace("\\", "Ñ") + line[fin_nombre:]
                    if line[(inicio_nombre-1)] == '"':
                        line = line[:inicio_nombre] + line[inicio_nombre:fin_nombre].replace("'", "*") + line[fin_nombre:]
                    if line[(inicio_nombre-1)] == "'":
                        line = line[:inicio_nombre] + line[inicio_nombre:fin_nombre].replace("'", "*") + line[fin_nombre:]
                
                # Reemplazar comillas simples por dobles en toda la línea
                line = line.replace("'", "\"")
                
                # Restaurar comillas simples en nombres de VJ
                line = line.replace("*","'")
                
                line = line.replace("Ñ"," ")
                
                # Cargar la línea corregida como objeto JSON
                data = json.loads(line)
                data_list.append(data)

            except json.JSONDecodeError as e:
                print(f"Error en línea: {line.strip()}")
                print(e)
            aux = aux + 1
            #if aux == 1000:
            #    aux = 0
            #    break

    return data_list


In [6]:
# 4.1. Ejecución de la función para cargar el archivo users_items

data_users_items = abrir_users_items(ruta_users_items)
df_users_items = pd.DataFrame(data_users_items)
df_users_items.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

In [7]:
# 3.2. Función para abrir el archivo users_reviews

def abrir_users_reviews(archivo):
    data_list = []
    aux = 0

    with gzip.open(archivo, 'rt', encoding='utf-8') as file:
        for line in file:
            # Reemplazar comillas simples por dobles en toda la línea
            
            line = line.strip()

            # Buscar todas las ocurrencias de 'review'
            ocurrencias_review = [i.start() for i in re.finditer("'review':", line)]
            # Reemplazar temporalmente las comillas simples en nombres de VJ por "*"
            fin_ultimo_review = len(line)-4
            for indice_review in ocurrencias_review:
                #print(index," ",indice_review," ",ocurrencias_review)
                inicio_review = indice_review + 11
                
                fin_review = (line.find("'funny'",inicio_review)-5)
            
                if fin_review > 0:
                    #print(line[indice_review:fin_review])
                    line = line[:inicio_review] + line[inicio_review:fin_review].replace('"','*') + line[fin_review:]
                    line = line[:inicio_review] + line[inicio_review:fin_review].replace("'",'*') + line[fin_review:]
                    line = line[:inicio_review] + line[inicio_review:fin_review].replace("\\", "Ñ") + line[fin_review:]
                else:
                    #print(line[indice_review:fin_ultimo_review])
                    line = line[:inicio_review] + line[inicio_review:fin_ultimo_review].replace('"','*') + line[fin_ultimo_review:]
                    line = line[:inicio_review] + line[inicio_review:fin_ultimo_review].replace("'",'*') + line[fin_ultimo_review:]
                    line = line[:inicio_review] + line[inicio_review:fin_ultimo_review].replace("\\", "Ñ") + line[fin_ultimo_review:]
            
            #print(line.find("'review'"))
            line = line.replace(": True,",': "True",')
            line = line.replace(": False,", ': "False",')
            line = line.replace("'", "\"")
            line = line.replace("*", "'")
            line = line.replace("Ñ", "")
            
            try:
                # Cargar la línea corregida como objeto JSON
                data = json.loads(line)
                data_list.append(data)
                
            except json.JSONDecodeError as e:
                print(f"Error en línea: {line.strip()}")
                print(e)
            #aux = aux + 1
            #if aux == 1000:
            #    aux = 0
            #    break

    return data_list


In [8]:
# 4.2. Ejecución de la función para cargar el archivo users_reviews

data_user_reviews = abrir_users_reviews(ruta_user_reviews)
df_user_reviews= pd.DataFrame(data_user_reviews)
df_user_reviews.columns

Index(['user_id', 'user_url', 'reviews'], dtype='object')

In [9]:
# 3.3. Función para abrir el archivo steam_games

def abrir_steam_games(archivo):
    data_list = []
    aux = 0
    
    with gzip.open(archivo, 'rt', encoding='utf-8') as file:
        for line in file:
            # Reemplazar comillas simples por dobles en toda la línea
            #print(line)
            line = line.strip()
            try:
                # Cargar la línea corregida como objeto JSON
                data = json.loads(line)
                data_list.append(data)
                
            except json.JSONDecodeError as e:
                print(f"Error en línea: {line.strip()}")
                print(e)
                #aux = aux + 1
                #if aux == 1000:
                #    aux = 0
                #    break

    return data_list


In [10]:
# 4.3. Ejecución de la función para cargar el archivo steam_games

data_steam_games = abrir_steam_games(ruta_steam_games)
df_steam_games = pd.DataFrame(data_steam_games)
df_steam_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

### 5. Exportar los DataFrames como archivos .csv

In [11]:
# 5. Exportar los DataFrames como archivos .csv

df_users_items.to_csv("users_items_extraido", index=False)
df_user_reviews.to_csv("users_reviews_extraido", index=False)
df_steam_games.to_csv("steam_games_extraido", index=False)

**FIN**